In [8]:
from web_scraper.web_scraper import CompendiumScraper
import pandas as pd
from selenium.webdriver.common.by import By
import psycopg2
import subprocess
import yaml
import sqlalchemy as sa

In [9]:
with open(r'c:\Users\jasre\Code\ws_config.yml', 'r') as yml:
    config = yaml.safe_load(yml)

In [18]:
engine_url = sa.URL.create(
    "postgresql",
    username=config['pgdb_user'],
    password=config['pgdb_pass'],
    host=config['pgdb_host'],
    database=config['pgdb_data']
)

In [ ]:
engine = sa.create_engine(engine_url, echo=True)

In [21]:
raw_abilities = pd.read_csv(config['datasets_dir'] + 'raw_abilities_2.csv')

In [23]:
raw_abilities.to_sql('raw_abilities', con=engine, if_exists='replace', index=False)

2023-08-20 09:56:54,239 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-08-20 09:56:54,241 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-08-20 09:56:54,246 INFO sqlalchemy.engine.Engine select current_schema()
2023-08-20 09:56:54,247 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-08-20 09:56:54,254 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-08-20 09:56:54,255 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-08-20 09:56:54,257 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-20 09:56:54,311 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

273

In [24]:
raw_abilities.to_sql('test_table', con=engine, if_exists='replace', index=False)

2023-08-20 10:06:07,545 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-08-20 10:06:07,573 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2023-08-20 10:06:07,574 INFO sqlalchemy.engine.Engine [cached since 553.3s ago] {'table_name': 'test_table', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2023-08-20 10:06:07,580 INFO sqlalchemy.engine.Engine 
CREATE TABLE test_table (
	char_name TEXT, 
	ability_name TEXT, 
	ability_id FLOAT(53), 
	main_target_hp_attacks BIGINT, 
	non_target_hp_attacks BIGINT, 
	hp_dmg_cap_up_per

273

In [3]:
conn = psycopg2.connect(
    dbname=config['pgdb_data'],
    user=config['pgdb_user'],
    password=config['pgdb_pass'],
    host=config['pgdb_host']
)

cursor = conn.cursor()

In [4]:
cursor.execute(
    """
    SELECT
        *
    FROM public.raw_abilities;
    """
)

results = cursor.fetchall()

In [5]:
results

[]

In [6]:
cursor.close()
conn.close()

# Running To Do
- Add something to account for Lann & Reynn's difference HP Dmg Cap based on enemy count
- Characters' High Armor shouldn't be parsed a second time when you check their JP rework. If they're already available in GL, their HA is their HA.
- Enna Kros' abilities need to be a special consideration. Everything should be tripled for Bold Maneuver, Atomic Impact, and Astral Annihilation. This can be left alone for the ability_df, but in dashboarding, make sure to note this.
- Add a `_data_pulled_at` column for each dataset.
- Parse follow ups for JP versions of characters (manual...)
- Check if the abilities that can exceed the MBRV Cap all have `*Can exceed MAX BRV Cap` in their descriptions.
- Fix the incorrect information that you manually identified.

# Ability DF Notes from Initial Run
- The logs say that Lann & Reynn's BT don't have stacks. This could be intended from my code, but I can't remember.
- BT effects for all of the "spoiler alert" characters are missing, even over in JP.
- 

# Recoding that needs to happen for ability names

In [2]:
renaming = {
    'Å': 'ō',
    'Ã±': 'ñ',
    'ÃŸ': 'ß',
    'Ã—': '×',
    'ï½ž': '～',
    'ï¼Ÿ': '? ',
    'â€“': '--',
    "â˜†": "☆"
}

In [3]:
cs = CompendiumScraper('\\Users\\jasre\\Code\\ws_config.yml')

2023-08-13 19:58:39,966 - web_scraper.web_scraper - ====================================================
2023-08-13 19:58:39,967 - web_scraper.web_scraper - Retrieved all main character links. Generating remaining links for each character.
2023-08-13 19:58:39,968 - web_scraper.web_scraper - ====================================================
2023-08-13 19:58:40,006 - web_scraper.web_scraper - self.character_dict_omnibus entry for ASTOS was successful
2023-08-13 19:58:40,026 - web_scraper.web_scraper - self.character_dict_omnibus entry for IRIS was successful
2023-08-13 19:58:40,048 - web_scraper.web_scraper - self.character_dict_omnibus entry for REKS was successful
2023-08-13 19:58:40,087 - web_scraper.web_scraper - self.character_dict_omnibus entry for QUINA was successful
2023-08-13 19:58:40,108 - web_scraper.web_scraper - self.character_dict_omnibus entry for RUBICANTE was successful
2023-08-13 19:58:40,129 - web_scraper.web_scraper - self.character_dict_omnibus entry for NEON was

In [4]:
ability_df_list = []

bt_effect_df_list = []

ha_cap_df_list = []

jp_list = []

for char_name in ['amarant', 'leonora', 'selhteus', 'kelger', 'weiss', 'jihlnabaat']:
    try:
        cs.generate_ability_dict(char_name)
    except Exception:
        jp_list.append(char_name)
        continue
    
    parsed_ability_df = cs.parse_ability_dict_to_df(char_name)

    ability_df_list.append(parsed_ability_df)

    bt_effect_df = cs.retrieve_hp_caps_from_bt(char_name, return_output=True)

    bt_effect_df_list.append(bt_effect_df)

    high_armor_cap_df = cs.retrieve_ha_hp_dmg_cap_up(char_name, return_output=True)

    ha_cap_df_list.append(high_armor_cap_df)

2023-08-13 19:58:43,102 - web_scraper.web_scraper - ====================================================
2023-08-13 19:58:43,102 - web_scraper.web_scraper - Generating ability dictionary for AMARANT
2023-08-13 19:58:43,103 - web_scraper.web_scraper - ====================================================
2023-08-13 19:58:48,364 - web_scraper.web_scraper - Unable to access abilities for AMARANT.
2023-08-13 19:58:48,364 - web_scraper.web_scraper - They might not be released to GL yet.
2023-08-13 19:58:48,365 - web_scraper.web_scraper - ====================================================
2023-08-13 19:58:48,365 - web_scraper.web_scraper - Parsing ability dict to df for AMARANT.
2023-08-13 19:58:48,366 - web_scraper.web_scraper - ====================================================
2023-08-13 19:58:48,367 - web_scraper.web_scraper - Couldn't find ability_dict for AMARANT.
2023-08-13 19:58:48,368 - web_scraper.web_scraper - ====================================================
2023-08-13 19:5

No ability_dict for amarant.
Run `generate_ability_dict('amarant')` first, then `parse_ability_dict_to_df()` should work for them.
Also, make sure you've properly specified whether you want the GL or JP version for both functions.


2023-08-13 19:59:00,026 - web_scraper.web_scraper - No stack slider found. Assuming amarant has a BT without stacks.
2023-08-13 19:59:00,056 - web_scraper.web_scraper - Retrieved BT info for AMARANT.
2023-08-13 19:59:00,058 - web_scraper.web_scraper - ====================================================
2023-08-13 19:59:00,059 - web_scraper.web_scraper - Retrieving High Armor info for AMARANT.
2023-08-13 19:59:00,060 - web_scraper.web_scraper - ====================================================
2023-08-13 19:59:10,737 - web_scraper.web_scraper - Could not find High Armor for AMARANT.
2023-08-13 19:59:10,738 - web_scraper.web_scraper - ====================================================
2023-08-13 19:59:10,738 - web_scraper.web_scraper - Generating ability dictionary for LEONORA
2023-08-13 19:59:10,739 - web_scraper.web_scraper - ====================================================


Unable to find High Armor info for Amarant.
They might not be released to GL yet.


2023-08-13 19:59:16,077 - web_scraper.web_scraper - Unable to access abilities for LEONORA.
2023-08-13 19:59:16,077 - web_scraper.web_scraper - They might not be released to GL yet.
2023-08-13 19:59:16,078 - web_scraper.web_scraper - ====================================================
2023-08-13 19:59:16,079 - web_scraper.web_scraper - Parsing ability dict to df for LEONORA.
2023-08-13 19:59:16,079 - web_scraper.web_scraper - ====================================================
2023-08-13 19:59:16,080 - web_scraper.web_scraper - Couldn't find ability_dict for LEONORA.
2023-08-13 19:59:16,081 - web_scraper.web_scraper - ====================================================
2023-08-13 19:59:16,081 - web_scraper.web_scraper - Retrieving BT info for LEONORA.
2023-08-13 19:59:16,082 - web_scraper.web_scraper - ====================================================


...

Unable to find High Armor info for Weiss.
They might not be released to GL yet.


2023-08-13 20:01:07,298 - web_scraper.web_scraper - Unable to access abilities for JIHLNABAAT.
2023-08-13 20:01:07,299 - web_scraper.web_scraper - They might not be released to GL yet.
2023-08-13 20:01:07,299 - web_scraper.web_scraper - ====================================================
2023-08-13 20:01:07,300 - web_scraper.web_scraper - Parsing ability dict to df for JIHLNABAAT.
2023-08-13 20:01:07,301 - web_scraper.web_scraper - ====================================================
2023-08-13 20:01:07,301 - web_scraper.web_scraper - Couldn't find ability_dict for JIHLNABAAT.
2023-08-13 20:01:07,302 - web_scraper.web_scraper - ====================================================
2023-08-13 20:01:07,302 - web_scraper.web_scraper - Retrieving BT info for JIHLNABAAT.
2023-08-13 20:01:07,303 - web_scraper.web_scraper - ====================================================


No ability_dict for jihlnabaat.
Run `generate_ability_dict('jihlnabaat')` first, then `parse_ability_dict_to_df()` should work for them.
Also, make sure you've properly specified whether you want the GL or JP version for both functions.


2023-08-13 20:01:18,984 - web_scraper.web_scraper - No stack slider found. Assuming jihlnabaat has a BT without stacks.
2023-08-13 20:01:19,008 - web_scraper.web_scraper - Retrieved BT info for JIHLNABAAT.
2023-08-13 20:01:19,009 - web_scraper.web_scraper - ====================================================
2023-08-13 20:01:19,009 - web_scraper.web_scraper - Retrieving High Armor info for JIHLNABAAT.
2023-08-13 20:01:19,010 - web_scraper.web_scraper - ====================================================
2023-08-13 20:01:29,720 - web_scraper.web_scraper - Could not find High Armor for JIHLNABAAT.


Unable to find High Armor info for Jihlnabaat.
They might not be released to GL yet.
